In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils import data
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.optim import lr_scheduler
from torch.optim import lr_scheduler
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import random_split

global cr
cr = 0.98


def normalization(x: list):
    M, m = np.max(x), np.min(x)
    for i in range(len(x)):
        x[i] = (x[i] - (M + m) / 2) / ((M - m) / 2)
    # x in [-1, 1]
    return M, m, x
def cal(x: list):
    mu, sigma = 0, 0
    for i in range(len(x)):
        mu += x[i]
    mu /= len(x)*1.0
    for i in range(len(x)):
        sigma += (x[i]-mu) ** 2
    sigma /= len(x)*1.0
    from math import sqrt
    for i in range(len(x)):
        x[i] = (x[i]-mu) / sqrt(sigma)
    return mu, sigma, x

def ArrNorm(x: np.ndarray, state='max-min'):
    assert isinstance(x, np.ndarray), "We need a list"
    
    if state is 'max-min':
        M_list, m_list, res = [], [], []
        for i in range(x.shape[0]):
            u = x[i].tolist()
            M, m, t = normalization(u)
            res.append(t)
            M_list.append(M)
            m_list.append(m)
        return M_list, m_list, np.array(res)
    elif state is 'standard':
        mu_list, sigma_list, res = [], [], []
        for i in range(x.shape[0]):
            u = x[i].tolist()
            m, s, t = cal(u)
            res.append(t)
            mu_list.append(m)
            sigma_list.append(s)
        return mu_list, sigma_list, res


def df2arr(x) -> np.ndarray:
    return np.array(x, dtype=np.float32)



# one train demo

class Try(nn.Module):
    def __init__(self, seq, batch_size, scale=0):
        super(Try, self).__init__()
        self.scale = scale
        self.seq = seq
        self.batch_size = batch_size
        self.linear = nn.Sequential(
            nn.Linear((self.seq+1)*12, (self.seq+1)*6),
            nn.Dropout(0.5),
            nn.Sigmoid(),

            nn.Linear((self.seq+1)*6, (self.seq+1)*6),
            nn.Dropout(0.5),
            nn.ReLU(inplace=True),

        )
        self.conv1 = nn.Sequential(
            # seq * 5 
            nn.Conv2d(1, 2, kernel_size=(3,3), padding=2, bias=False), # (seq+2) * 7
            nn.BatchNorm2d(2),
            nn.ReLU(inplace=True),

            nn.Conv2d(2, 2, kernel_size=(3,3), padding=1, bias=False), # (seq+2) * 7
            nn.BatchNorm2d(2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,2), stride=1), # (seq+1) * 6
            
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(1, 2, kernel_size=(2,2), padding=0, bias=False), # seq * 5
            nn.BatchNorm2d(2),
            nn.ReLU(inplace=True),

            nn.Conv2d(2, 1, kernel_size=(1,1), padding=0, bias=True), # seq * 5
            nn.ReLU(inplace=True)

        )

    def forward(self, x):
        out = self.conv1(x)
        out = out.view(out.size()[0], -1)
        out = self.linear(out)
        # print(out.shape)
        with torch.no_grad():
            out = out.reshape(self.batch_size, 1, self.seq+1, 6)
        out = self.conv1(out)
        assert out.shape==x.shape, "Shape Unequal Error."
        return out + x
    
    



In [ ]:
# !pip show torch
# !pip show pandas
# !pip show matplotlib
# !pip show tqdm
# !pip show scipy

In [ ]:
import math
import numpy as np
from scipy import stats
def rsquared(x, y): 
#     print(x.shape)
    length = x.shape[-1]
    assert x.shape==y.shape, "Unequal Shape Error"
    r, x, y = [], x.detach(), y.detach()
    x, y = x.mean(dim=[0,1], keepdim=False), y.mean(dim=[0,1], keepdim=False)
#     print(x.shape)
    for i in range(length):
        _, _, r_value, _, _ = stats.linregress(x[:,i].detach().numpy(), y[:,i].detach().numpy()) 
        r.append(r_value ** 2)
    return r

In [ ]:
class NET(nn.Module):
    def __init__(self, seq, batch_size, ablation_scale=1.0):
        super(NET, self).__init__()
        self.seq = seq
        self.batch_size = batch_size
        self.scale = ablation_scale
        self.input_size = self.hidden_size = seq
        
        self.is_directional = False
        self.num_direction = 2 if self.is_directional else 1
        self.num_layers = 1
        
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, dropout=0.5, bidirectional=self.is_directional, batch_first=False)
        
        self.decouple = nn.Sequential(
            nn.Linear(self.seq*5,self.seq*5),  # linear decouple
            nn.Dropout(0.5),
#             nn.Sigmoid(),   # use sigmoid to enhance unlinear regression may cause data shifting
            nn.Linear(self.seq*5, self.seq*5),     # unlinear pool
        )
        self.conv = nn.Sequential(
            # seq * 5 
            nn.Conv2d(1, 2, kernel_size=(3,3), padding=1, bias=False), # seq * 5
            nn.BatchNorm2d(2),
            nn.ReLU(inplace=True),

            nn.Conv2d(2, 2, kernel_size=(1,1), padding=0, bias=False),  # seq * 5
            nn.BatchNorm2d(2),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(2, 1, kernel_size=(3,3), padding=1, bias=True), # seq * 5
            nn.ReLU(inplace=True),
        )
    
    def reset(self, scale=1.0):
        self.ablatiion_scale = scale

    def forward(self, x):
        x = x.squeeze()
        
        h_0 = torch.randn(self.num_direction*self.num_layers, self.seq, self.hidden_size).cuda()
        c_0 = torch.randn(self.num_direction*self.num_layers, self.seq, self.hidden_size).cuda()
        pred, *_ = self.lstm(x, (h_0, c_0))
        pred = pred.flatten(1)
        out1 = self.decouple(pred)
        with torch.no_grad():
            out1 = out1.reshape(-1, 1, self.seq, 5)
        out2 = self.conv(out1)
        assert out1.shape==out2.shape, "Shape Unequal Error."
        return self.scale * out1 + out2

In [ ]:
excel = pd.read_excel('/kaggle/input/a32-data/A32.xlsx', header=None)
excel.shape

sp = [1486, 2972, 4458]
station_1 = excel.iloc[1:sp[0]+1,1:6]
station_2 = excel.iloc[sp[0]+1:sp[1]+1,1:6]
standard = excel.iloc[sp[1]+1:sp[2]+1,1:6]

station_1 = df2arr(station_1)
station_2 = df2arr(station_2)
standard = df2arr(standard)
station_1.shape, station_2.shape, standard.shape

In [ ]:
s1_minus_sd = station_1 - standard
s2_minus_sd = station_2 - standard
s1_div_sd = station_1 / standard
s2_div_sd = station_2 / standard

s1_minus_sd.shape, s2_minus_sd.shape

In [ ]:
s1_M, s1_m, s1 = ArrNorm(station_1, 'standard')
s2_M, s2_m, s2 = ArrNorm(station_2, 'standard')

import math
import numpy as np
from scipy import stats
def rsquared(x, y): 
    _, _, r_value, _, _ = stats.linregress(x.detach().cpu().numpy(), y.detach().cpu().numpy()) 
    return r_value**2

In [ ]:
def GetDataset(input_arr: list, output_arr: list, seq: int):
    assert(len(input_arr)==len(output_arr)), "Different size of input and output!"
    Input = []
    Output = []
    for i in range(len(input_arr)-seq):
        Input.append(input_arr[i:i+seq][:])
        Output.append(output_arr[i:i+seq][:])
    return torch.tensor(Input, dtype=torch.float32), torch.tensor(Output, dtype=torch.float32)

        
def load_array(data_arrays, batch_size, is_train=True):
    # data-iter
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

s1_minus_sd.shape

In [ ]:
!pip install loguru

In [ ]:
# global sequence, batch_size
# batch_size = 4
# sequence = 6

# global lr, num_epoch
# lr, num_epoch = 0.000001, 100
# status_list = ['MINUS', 'DIVIDE']

from loguru import logger


status_list = ["MINUS"]
name_list = ["TRY", "NET"]
global record
record_R = []
record_con = []
weights = []
cnt = 0


with open(r'/kaggle/working/log.txt', 'w', encoding='utf-8') as f:
    f.write('START TRAINING...\n')


FILE = open('/kaggle/working/log.txt', "w", encoding='utf-8')

def train_test(name, status, batch_size, sequence, lr, num_epoch, ga, scale):
    global cnt
    cnt += 1
    training_scale_ = scale
    string = "&sequence=" + str(sequence) + "&scale=" + str(scale) 
    weights.append(string)
    
    Input_Data_1, Output_Data_1 = GetDataset(s1, s1_minus_sd, sequence) if status=="MINUS" else GetDataset(s1,s1_div_sd, sequence)
    Input_Data_2, Output_Data_2 = GetDataset(s2, s2_minus_sd, sequence) if status=="MINUS" else GetDataset(s2,s2_div_sd, sequence)
    Input_Data_1.shape, Input_Data_2.shape

    Input_Data_1 = Input_Data_1.unsqueeze(1)
    Output_Data_1 = Output_Data_1.unsqueeze(1)
    Input_Data_2 = Input_Data_2.unsqueeze(1)
    Output_Data_2 = Output_Data_2.unsqueeze(1)
    Input_Data_1.shape, Input_Data_1.shape


    data_tot_1 = torch.utils.data.TensorDataset(Input_Data_1, Output_Data_1)
    data_tot_2 = torch.utils.data.TensorDataset(Input_Data_2, Output_Data_2)
    train_size = int(Input_Data_1.shape[0] * cr)
    test_size = Input_Data_1.shape[0] - train_size
    train_set_1, test_set_1 = random_split(data_tot_1,[train_size,test_size],
                                        torch.Generator().manual_seed(0))
    train_set_2, test_set_2 = random_split(data_tot_2,[train_size,test_size],
                                        torch.Generator().manual_seed(0))




    Data_Iter_1 = DataLoader(dataset=train_set_1, batch_size=batch_size, shuffle=True, drop_last=True)
    Data_Iter_2 = DataLoader(dataset=train_set_2, batch_size=batch_size, shuffle=True, drop_last=True)

    x_plt, train_loss_plt = [], []


    net = Try(batch_size=batch_size, seq=sequence).cuda() if name=="TRY" else NET(batch_size=batch_size, seq=sequence, ablation_scale=scale).cuda()
    Loss = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=ga)

    def Iter(num_epoch):
        cnt = 0
        while cnt < num_epoch:
            yield cnt
            cnt += 1

    print("\nStart Training with condition: "+string)
    print("Round 1...")
    for epoch in range(num_epoch):
        epoch_start_time = time.time()
        train_loss = 0.0
        net.train()
        for i, use in enumerate(tqdm(Data_Iter_1)):
            optimizer.zero_grad()
            train_pred = net(use[0].cuda())    # use[0].cuda()

            batch_loss = Loss(train_pred, use[1].cuda())   # use[1].cuda()
            batch_loss.backward()
            optimizer.step()

            train_loss += batch_loss.item()

        train_loss = train_loss / train_size
        x_plt.append(epoch+1)
        train_loss_plt.append(train_loss)
        print("Round-1 --- [%2d|%2d] %.2f(s) Train_Loss=%.6f "%\
                (epoch+1,num_epoch,time.time()-epoch_start_time,train_loss),end='')
        scheduler.step()  

    plt.figure(1)
    plt.plot(x_plt,train_loss_plt,'rs-',label='all_train_loss')
    plt.show()

    print("Round 2...")
    for epoch in range(num_epoch):
        epoch_start_time = time.time()
        train_loss = 0.0
        net.train()
        for i, use in enumerate(tqdm(Data_Iter_2)):
            optimizer.zero_grad()
            train_pred = net(use[0].cuda())    # use[0].cuda()

            batch_loss = Loss(train_pred, use[1].cuda())   # use[1].cuda()
            batch_loss.backward()
            optimizer.step()

            train_loss += batch_loss.item()

        train_loss = train_loss / train_size
        x_plt.append(epoch+num_epoch+1)
        train_loss_plt.append(train_loss)
        print("Round 2 --- [%2d|%2d] %.2f(s) Train_Loss=%.6f "%\
                (epoch+1,num_epoch,time.time()-epoch_start_time,train_loss),end='')
        scheduler.step() 


    plt.figure(2)
    plt.plot(x_plt,train_loss_plt,'rs-',label='all_train_loss')
    plt.show()

    torch.save(net.state_dict(), '/kaggle/working/'+string+".pt")
    print("Parameters Saved.")


    Test_Iter_1 = DataLoader(dataset=test_set_1, batch_size=len(test_set_1), shuffle=False, drop_last=True)
    Test_Iter_2 = DataLoader(dataset=test_set_2, batch_size=len(test_set_2), shuffle=False, drop_last=True)
    model = NET(batch_size=1, seq=sequence, ablation_scale=0.5)
    model.load_state_dict(torch.load('/kaggle/working/'+string+'.pt'))
    model = model.cuda()
    model.eval()
    
    
    
    scale = 0.5
    model.reset(scale)
    R_1, R_2 = [], []
    with torch.no_grad():
        for i, use in enumerate(Test_Iter_1):
            pred = model(use[0].cuda())
            pred = pred.reshape(-1, 5)
            use[1] = use[1].reshape(-1, 5)
            assert pred.shape == use[1].shape, 'unequal shape error'
            for i in range(pred.shape[1]):
                R_1.append(rsquared(pred[i], use[1][i]))
            assert len(R_1) == 5, 'Err'
        for i, use in enumerate(Test_Iter_2):
            pred = model(use[0].cuda())
            pred = pred.reshape(-1, 5)
            use[1] = use[1].reshape(-1, 5)
            assert pred.shape == use[1].shape, 'unequal shape error'
            for i in range(pred.shape[1]):
                R_2.append(rsquared(pred[i], use[1][i]))
            assert len(R_2) == 5, 'Err'
            
            print('Guidance Scale = {0}, R_1 = {1}, R_2 = {2}'.format(scale, R_1, R_2))
    
    def find_scale(idx: int):
        # feature_index: idx
        model = NET(batch_size=1, seq=sequence, ablation_scale=0.5)
        model.load_state_dict(torch.load('/kaggle/working/'+string+'.pt'))
        model = model.cuda()
        model.eval()

        Test_Iter_1 = DataLoader(dataset=test_set_1, batch_size=len(test_set_1), shuffle=False, drop_last=True)
        Test_Iter_2 = DataLoader(dataset=test_set_2, batch_size=len(test_set_2), shuffle=False, drop_last=True)

        r = .0
        r12 = []
        best_scale = .0

        for scale in np.arange(.0, 10.0, 0.1):
            model.reset(scale)
            with torch.no_grad():
                for i, use in enumerate(Test_Iter_1):
                    pred = model(use[0].cuda())
                    pred = pred.reshape(-1, 5)
                    use[1] = use[1].reshape(-1, 5)
                    assert pred.shape == use[1].shape, 'unequal shape error'
                    r1 = rsquared(pred[idx], use[1][idx])
                for i, use in enumerate(Test_Iter_2):
                    pred = model(use[0].cuda())
                    pred = pred.reshape(-1, 5)
                    use[1] = use[1].reshape(-1, 5)
                    assert pred.shape == use[1].shape, 'unequal shape error'
                    r2 = rsquared(pred[idx], use[1][idx])

                r_ = r1 + r2
                if r_ >= r:
                    r = r_
                    best_scale = scale
                    r12 = [r1, r2]
        return r, best_scale, r12
        
    r = []
    best_scale = []
    r12 = []
    for i in range(5):
        r_, scale_, r12_ = find_scale(i)
        r.append(r_ / 2)
        r12.append(r12_)
        best_scale.append(scale_)
        
    logger.info('training scale: ')
    logger.info(training_scale_)
    logger.info('R List: ')
    logger.info(r)
    logger.info('Best Scale List: ')
    logger.info(best_scale)
    logger.info('\n')
    
    print('training scale:\n', training_scale_)
    print('R List:\n', r)
    print('Best Scale List:\n', best_scale)
    print('Two of R in List:\n', r12)
    
    FILE.write('training scale: ')
    FILE.write('\n')
    FILE.write(str(training_scale_))
    FILE.write('\n')
    FILE.write('R List: ')
    FILE.write('\n')
    FILE.write(str(r))
    FILE.write('\n')
    FILE.write('Best Scale List: ')
    FILE.write('\n')
    FILE.write(str(best_scale))
    FILE.write('\n')
    FILE.write('Two of R in List: ')
    FILE.write('\n')
    FILE.write(str(r12))
    FILE.write('\n')
    FILE.write('-'*90)
    FILE.write('\n')
    



In [ ]:
name_list = ["NET"]
status = "MINUS"
num_epoch = 3000
ga = 0.90
lr = 0.0003

for train_scale in np.arange(-10., 10., .5):
    for batch_size in [16]:
        for sequence in [5]:
            for name in name_list:
                train_test(name, status, batch_size, sequence, lr, num_epoch, ga, train_scale)



In [ ]:
FILE.close()